# qiskit_alt

This python package (re-)implements some features of Qiskit in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared one demonstration&mdash;performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator. We will see that the `qiskit_alt` implementation is much more performant.

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
import qiskit_alt  # The message below is from Julia

  Activating project at `~/myrepos/quantum_repos/qiskit_alt`


We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
qiskit_alt.Main.zeros(10)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
fermi_op = qiskit_alt.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.qubit_hamiltonian(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention, so we can sort the result to  

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature`.

First we benchmark qiskit-nature, which records the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 7.69 ms
geometry=h2_geometry, basis='631g' 107.52 ms
geometry=h2_geometry, basis='631++g' 569.66 ms
geometry=h2o_geometry, basis='sto3g' 687.79 ms
geometry=h2o_geometry, basis='631g' 19075.31 ms


[7.689325185492635,
 107.52431633882225,
 569.6588844060898,
 687.7901777625084,
 19075.30520996079]

Next we benchmark qiskit_alt, which records the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.82 ms
geometry=h2_geometry, basis='631g' 5.70 ms
geometry=h2_geometry, basis='631++g' 27.46 ms
geometry=h2o_geometry, basis='sto3g' 25.30 ms
geometry=h2o_geometry, basis='631g' 628.43 ms


[0.8164084982126951,
 5.700296815484762,
 27.4592365603894,
 25.295126484706998,
 628.4349207766354]

We compare the relative performance.

In [10]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[9.418477639963726,
 18.86293289969291,
 20.74561989927412,
 27.19062022395241,
 30.3536684218424]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`. We have not yet organized benchmark data for this.